In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
channel = 'D-4'
fname = f'dataset/smap_data/ad_{channel}_all_sensors.csv'
df_sensor = pd.read_csv(fname)
df_sensor

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,channel,label,cat_label,anomaly,normal,set_type
0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,normal,0,False,True,adj_w
1,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,normal,0,False,True,testset
2,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,normal,0,False,True,testset
3,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,normal,0,False,True,testset
4,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,normal,0,False,True,testset
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8468,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,anomaly,1,True,False,trainset
8469,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,anomaly,1,True,False,trainset
8470,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D-4,anomaly,1,True,False,trainset
8471,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,D-4,anomaly,1,True,False,adj_w


In [27]:
def sliding_window(inputs, target, indices, window_size):
    """
    Create an improved sliding window function for a 2D array.

    Each input sample is transformed into a 1D array, arranged such that
    the elements go through each column of the window.

    :param inputs: 2D array of shape (samples, sensors)
    :param target: 1D array of labels
    :param indices: List of indices to create windows from
    :param window_size: Size of the sliding window
    :return: Tuple of two lists containing windows for inputs and target
    """

    valid_indices = [index for index in indices if index >= window_size - 1]
    n_samples = len(valid_indices)
    sensor_size = inputs.shape[1]
    
    x_slide = torch.zeros((n_samples,sensor_size*window_size))
    if np.ndim(target) == 1:
        y_slide = torch.empty(n_samples)
    else:
        y_slide = torch.empty(n_samples,target.shape[1])

    for i,index in enumerate(valid_indices):
        # Initialize empty window
        input_window = []

        # Populate the window by column
        for sensor_idx in range(inputs.shape[1]):
            column = []
            for window_idx in range(window_size):
                column.append(inputs[index - window_idx, sensor_idx])
            input_window.extend(column)

        # target_window = target[index]

        x_slide[i] = torch.tensor(input_window)
        y_slide[i] = torch.tensor(target[index])

    return x_slide, y_slide

In [28]:
#### Vanila example
inputs = np.array([[0,1,2,3,4,5],[1,2,0,0,0,0],[2,2,0,0,0,0],[3,1,2,3,4,5],\
                   [4,2,0,0,0,0],[5,2,0,0,0,0],[6,1,2,3,4,5],[7,1,2,3,4,5]])
targets = np.array([1,2,3,4,5,6,7,8])

ws= 2
idx = [0,3,5]

x,y = sliding_window(inputs,targets,idx,ws)
x,y

(tensor([[3., 2., 1., 2., 2., 0., 3., 0., 4., 0., 5., 0.],
         [5., 4., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 tensor([4., 6.]))

In [29]:
set_type = 'trainset'
mask = df_sensor.set_type == set_type
train_idx = df_sensor[mask].index.values
inputs = df_sensor.loc[:,[str(i) for i in range(25)]].to_numpy()
labels = df_sensor.loc[:,['anomaly','normal']].to_numpy()
x,y = sliding_window(inputs,labels,train_idx,3)
x

tensor([[-1., -1., -1.,  ...,  0.,  0.,  0.],
        [-1., -1., -1.,  ...,  0.,  0.,  0.],
        [-1., -1., -1.,  ...,  0.,  0.,  0.],
        ...,
        [ 1.,  1.,  1.,  ...,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  ...,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  ...,  0.,  0.,  0.]])

# Checking loader functionality

In [1]:
from source.loaders.loadBinSetV4_5 import data
from source.loaders.loaderV4_5 import loader
from source._helpersV4 import logSave,load_parameters
import numpy as np
import pandas as pd

In [2]:
p_path = 'experiments/smap_100/smap_p.json'
p = load_parameters(p_path)
channel = 'D-4'
fname = f'ad_{channel}_all_sensors.csv'
p['data_path'] += fname
dataset = loader(p,'adj_w',float)

In [3]:
dataset.targets

array([[ True, False],
       [ True, False],
       [ True, False],
       ...,
       [False,  True],
       [False,  True],
       [False,  True]])

In [4]:
dataset.testset()

(tensor([[-1.0000, -1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-1.0000, -1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-1.0000, -1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 1.0000,  1.0000,  0.9982,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.0000,  1.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor([[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [0., 1.],
         [0., 1.],
         [0., 1.]]))